In [ ]:
pip install python-telegram-bot yt-dlp

In [ ]:
pip install python-telegram-bot --upgrade

In [ ]:
pip install nest_asyncio

In [ ]:
import os
import logging
from yt_dlp import YoutubeDL
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Application, CommandHandler, MessageHandler, CallbackQueryHandler, filters, CallbackContext
import asyncio
import nest_asyncio

# Appliquer nest_asyncio pour permettre les boucles d'événements imbriquées
nest_asyncio.apply()

# Configurer le logging pour le suivi des erreurs
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

# Fonction pour télécharger la vidéo avec yt-dlp et renvoyer l'URL ou envoyer le fichier à l'utilisateur
def download_video_with_ytdlp(url, only_audio, quality):
    download_path = os.path.join(os.path.expanduser('~'), 'Downloads', 'Youtube_Downloads')
    os.makedirs(download_path, exist_ok=True)

    # Choix du format selon la qualité et l'option audio
    if only_audio:
        format_option = 'bestaudio'
    else:
        format_option = 'bestvideo' if quality == 'high' else 'worstvideo'

    ydl_opts = {
        'outtmpl': f'{download_path}/%(title)s.%(ext)s',
        'format': format_option,
        'quiet': False
    }

    try:
        with YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        # Retourne le chemin du fichier téléchargé
        filename = os.path.join(download_path, f"{ydl.prepare_filename(ydl.extract_info(url))}")
        return filename
    except Exception as e:
        return f"Erreur lors du téléchargement : {str(e)}"

# Commande pour démarrer le téléchargement
async def start(update: Update, context: CallbackContext) -> None:
    await update.message.reply_text("Envoyez-moi l'URL de la vidéo YouTube que vous souhaitez télécharger.")

# Fonction pour gérer les URL de vidéo
async def handle_url(update: Update, context: CallbackContext) -> None:
    url = update.message.text
    context.user_data["url"] = url  # Sauvegarder l'URL pour une utilisation ultérieure
    await update.message.reply_text(f"Je vais télécharger la vidéo à partir de : {url}\nChoisissez l'option suivante :")
    
    # Créer un clavier avec des options pour choisir vidéo ou audio
    keyboard = [
        [InlineKeyboardButton("1. Télécharger vidéo MP4", callback_data='video')],
        [InlineKeyboardButton("2. Télécharger audio MP3", callback_data='audio')]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    await update.message.reply_text("Choisissez 1 pour la vidéo ou 2 pour l'audio.", reply_markup=reply_markup)

# Fonction pour gérer la sélection vidéo ou audio
async def handle_media_type(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    await query.answer()
    
    url = context.user_data.get("url", "")
    if not url:
        await query.message.reply_text("D'abord, envoyez-moi l'URL de la vidéo.")
        return

    if query.data == 'video':
        context.user_data["media_type"] = 'video'
    elif query.data == 'audio':
        context.user_data["media_type"] = 'audio'
    
    # Après avoir choisi vidéo ou audio, proposer les choix de qualité
    keyboard = [
        [InlineKeyboardButton("1. Haute qualité", callback_data='high')],
        [InlineKeyboardButton("2. Basse qualité", callback_data='low')]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    await query.message.reply_text("Choisissez la qualité:", reply_markup=reply_markup)

# Fonction pour gérer la qualité (haute ou basse)
async def handle_quality(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    await query.answer()

    media_type = context.user_data.get("media_type", None)
    quality = 'high' if query.data == 'high' else 'low'
    url = context.user_data.get("url", "")

    if not media_type:
        await query.message.reply_text("Choisissez d'abord si vous voulez télécharger une vidéo ou de l'audio.")
        return

    # Appel à la fonction de téléchargement
    only_audio = True if media_type == 'audio' else False
    result = download_video_with_ytdlp(url, only_audio, quality)
    
    # Si le fichier est téléchargé avec succès, envoyez-le à l'utilisateur
    if os.path.exists(result):
        if only_audio:
            await query.message.reply_audio(audio=open(result, 'rb'))
        else:
            await query.message.reply_video(video=open(result, 'rb'))
    else:
        await query.message.reply_text(result)

async def main():
    # API Token directement intégré dans le code
    api_token = '************************************'  # Remplacez par votre propre API Token

    application = Application.builder().token(api_token).build()

    # Commandes
    application.add_handler(CommandHandler("start", start))

    # Gestion des messages avec des URL
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_url))
 
    # Gestion des choix vidéo/audio via CallbackQueryHandler
    application.add_handler(CallbackQueryHandler(handle_media_type, pattern='^(video|audio)$'))

    # Gestion des choix de qualité via CallbackQueryHandler
    application.add_handler(CallbackQueryHandler(handle_quality, pattern='^(high|low)$'))

    # Lancer le bot
    await application.run_polling()

if __name__ == '__main__':
    asyncio.run(main())  # Utiliser asyncio.run() pour démarrer le bot dans une boucle existante